In [1]:
import math
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import spsolve

def solve_schrodinger2D(bcs, L, m, potential):
    # create grid
    h = L / (m-1)
    x = np.linspace(0, L, m)
    y = np.linspace(0, L, m)
    X, Y = np.meshgrid(x[1:m-1], y[1:m-1])
    
    # get potential
    V = potential(X, Y)
    
    # get forcing
    f = np.zeros((m-2, m-2))
    
    # build Hamiltonian
    diag = np.ones(m-2)
    diags = np.array([diag, -2*diag, diag])
    D = sparse.spdiags(diags, np.array([-1, 0, 1]), m-2, m-2) / h**2
    T = sparse.kronsum(D, D)
    C = sparse.diags(V.reshape((m-2)**2), (0))
    H = T - C
    
    # enforce the four boundary/gauge conditions
    n = bcs.shape[1]
    fs = np.repeat(f[:, :, np.newaxis], n, axis=2)
    fs[0, :, :] += -bcs[1:m-1, :] / (h**2)
    fs[:, -1, :] += -bcs[m:2*m-2, :] / (h**2)
    fs[-1, :, :] += -np.flip(bcs[2*m-1:3*m-3, :], axis=0) / (h**2)
    fs[:, 0, :] += -np.flip(bcs[3*m-2:4*m-4, :], axis=0) / (h**2)
    fs = np.reshape(fs, [(m-2)**2, n])
    
    # solve for the solution
    sols = spsolve(H, fs)
    sols = np.reshape(sols, [m-2, m-2, n])
    us = np.zeros([m, m, n])
    us[1:m-1, 1:m-1, :] = sols
    us[0, :, :] = bcs[0:m, :]
    us[:, -1, :] = bcs[m-1:2*(m-1)+1, :]
    us[-1, :, :] = np.flip(bcs[2*(m-1):3*(m-1)+1, :], axis=0)
    us[:, 0, :] = np.flip(np.vstack([bcs[3*(m-1):4*(m-1), :], bcs[0, :]]), axis=0)
    
    return us

In [2]:
from karhunen_loeve import general_radial_kle
import sys
sys.path.append("..")
from helper import hexagonal_potential

potential = lambda x, y: hexagonal_potential(x, y, L=L, mag=1e3)

# Schrödinger equation parameters
L = 1

# number of samples
n = 500

# width of radial kernel
kernel_widths = [1e-2, 2.5e-2, 5e-2, 7.5e-2, 1e-1, 2.5e-1, 5e-1, 7.5e-1, 1.0, 2.5, 5.0, 7.5, 10.0]

# mesh sizes for forcing and solution
mesh_sizes = [50, 100, 200, 300, 400, 500]

In [ ]:
import h5py

# generate exponential KLE forcing and solutions for heat equation
file = h5py.File("schrodinger_exponentialperiodicKLE.hdf5", "a")
for m in mesh_sizes:
    print(f"Mesh Size: {m}")
    
    mesh_grp = file.require_group(f"mesh{m}")
    boundary_mesh = np.linspace(0, 4*L, 4*(m-1)+1)
    x_mesh = np.linspace(0, L, m)
    y_mesh = np.linspace(0, L, m)
    mesh_grp.attrs["input_mesh"] = boundary_mesh
    mesh_grp.attrs["output_mesh"] = (x_mesh, y_mesh)
    
    modes = m**2
    for kernel_width in kernel_widths:
        kernel_width_grp = mesh_grp.require_group(f"kernelwidth{kernel_width}")
        kernel = lambda x: np.exp(-2*np.sin(math.pi*np.abs(x)/(4*L))**2/kernel_width**2)
        fs = general_radial_kle((boundary_mesh,), n, modes, f=kernel).T
        fs_dset = kernel_width_grp.create_dataset("forcings", data=fs)

        us = solve_schrodinger2D(fs.T, L, m, potential).reshape(m**2, -1).T
        us_dset = kernel_width_grp.create_dataset("solutions", data=us)

file.close()
del file

Mesh Size: 50
Mesh Size: 100
Mesh Size: 200
Mesh Size: 300
